In [4]:
%matplotlib inline
import os, sys
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd
import torch
import torchvision
import cv2

sys.path.append(os.path.dirname(os.path.abspath(os.getcwd())))
from slomo import flownet as fl
from data import goes16s3
from tools import utils, inference_tools

from datetime import datetime
import xarray as xr
import scipy
import math
from skimage.measure import compare_ssim

font = {'family' : 'times',
        'size'   : 16}

matplotlib.rc('font', **font)



# Figures

##  1: Dataset Examples

##  2: Network Architecture

Network architecture as shown in slomo, including the multi-variate case. <br>
Flow and Intermediate Interpolation Networks. Can we use example input and flow images to represent it? I think so.

##  3: Optical flows of a test example
1. I0, I1, and It
2. Flows: F_01, F_10, F_01_delta, F_10_delta
3. Visible: V0 and V1
4. Difference between I0 and I1

## Figure 4: Time dependent errors
1. Linear interpolation
2. SloMo
3. MV-SloMo

# Tables
## 1: Overall and Per band errors
1, 3, 5, and 8 band experiments

In [5]:
# Download the required data
year = 2018
# Noreaster

days = [datetime(year, 3, 4).timetuple().tm_yday, # Noreaster
        datetime(year, 10, 8).timetuple().tm_yday,] # hurricane michael

n_channels = 8
noaa = goes16s3.NOAAGOESS3(channels=range(1,n_channels+1), save_directory='/raid/tj/GOES/TEST/')
for d in days:
    noaa.download_day(2018, d)

writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/12/00/RadM1/OR_ABI-L1b-RadM1-M3C01_G16_s20182811200279_e20182811200337_c20182811200377.nc
writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/12/01/RadM1/OR_ABI-L1b-RadM1-M3C01_G16_s20182811201279_e20182811201337_c20182811201378.nc
writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/12/02/RadM1/OR_ABI-L1b-RadM1-M3C01_G16_s20182811202279_e20182811202337_c20182811202379.nc
writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/12/03/RadM1/OR_ABI-L1b-RadM1-M3C01_G16_s20182811203279_e20182811203337_c20182811203377.nc
writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/12/04/RadM1/OR_ABI-L1b-RadM1-M3C01_G16_s20182811204279_e20182811204337_c20182811204378.nc
writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/12/05/RadM1/OR_ABI-L1b-RadM1-M3C01_G16_s20182811205279_e20182811205337_c20182811205379.nc
writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/12/06/RadM1/OR_ABI-L1b-RadM1-M3C01_G16_s20182811206279_e20182

writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/12/56/RadM1/OR_ABI-L1b-RadM1-M3C01_G16_s20182811256279_e20182811256337_c20182811256379.nc
writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/12/57/RadM1/OR_ABI-L1b-RadM1-M3C01_G16_s20182811257279_e20182811257337_c20182811257378.nc
writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/12/58/RadM1/OR_ABI-L1b-RadM1-M3C01_G16_s20182811258279_e20182811258337_c20182811258377.nc
writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/12/59/RadM1/OR_ABI-L1b-RadM1-M3C01_G16_s20182811259279_e20182811259337_c20182811259378.nc
writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/12/00/RadM1/OR_ABI-L1b-RadM1-M3C02_G16_s20182811200279_e20182811200337_c20182811200373.nc
writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/12/01/RadM1/OR_ABI-L1b-RadM1-M3C02_G16_s20182811201279_e20182811201337_c20182811201372.nc
writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/12/02/RadM1/OR_ABI-L1b-RadM1-M3C02_G16_s20182811202279_e20182

writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/12/52/RadM1/OR_ABI-L1b-RadM1-M3C02_G16_s20182811252279_e20182811252337_c20182811252373.nc
writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/12/53/RadM1/OR_ABI-L1b-RadM1-M3C02_G16_s20182811253279_e20182811253337_c20182811253373.nc
writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/12/54/RadM1/OR_ABI-L1b-RadM1-M3C02_G16_s20182811254279_e20182811254337_c20182811254374.nc
writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/12/55/RadM1/OR_ABI-L1b-RadM1-M3C02_G16_s20182811255279_e20182811255337_c20182811255370.nc
writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/12/56/RadM1/OR_ABI-L1b-RadM1-M3C02_G16_s20182811256279_e20182811256337_c20182811256371.nc
writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/12/57/RadM1/OR_ABI-L1b-RadM1-M3C02_G16_s20182811257279_e20182811257337_c20182811257372.nc
writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/12/58/RadM1/OR_ABI-L1b-RadM1-M3C02_G16_s20182811258279_e20182

writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/12/48/RadM1/OR_ABI-L1b-RadM1-M3C03_G16_s20182811248279_e20182811248337_c20182811248380.nc
writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/12/49/RadM1/OR_ABI-L1b-RadM1-M3C03_G16_s20182811249279_e20182811249337_c20182811249381.nc
writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/12/50/RadM1/OR_ABI-L1b-RadM1-M3C03_G16_s20182811250279_e20182811250337_c20182811250381.nc
writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/12/51/RadM1/OR_ABI-L1b-RadM1-M3C03_G16_s20182811251279_e20182811251337_c20182811251381.nc
writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/12/52/RadM1/OR_ABI-L1b-RadM1-M3C03_G16_s20182811252279_e20182811252337_c20182811252384.nc
writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/12/53/RadM1/OR_ABI-L1b-RadM1-M3C03_G16_s20182811253279_e20182811253337_c20182811253380.nc
writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/12/54/RadM1/OR_ABI-L1b-RadM1-M3C03_G16_s20182811254279_e20182

writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/12/44/RadM1/OR_ABI-L1b-RadM1-M3C04_G16_s20182811244279_e20182811244337_c20182811244369.nc
writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/12/45/RadM1/OR_ABI-L1b-RadM1-M3C04_G16_s20182811245279_e20182811245337_c20182811245369.nc
writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/12/46/RadM1/OR_ABI-L1b-RadM1-M3C04_G16_s20182811246279_e20182811246337_c20182811246369.nc
writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/12/47/RadM1/OR_ABI-L1b-RadM1-M3C04_G16_s20182811247279_e20182811247337_c20182811247370.nc
writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/12/48/RadM1/OR_ABI-L1b-RadM1-M3C04_G16_s20182811248279_e20182811248337_c20182811248368.nc
writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/12/49/RadM1/OR_ABI-L1b-RadM1-M3C04_G16_s20182811249279_e20182811249337_c20182811249370.nc
writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/12/50/RadM1/OR_ABI-L1b-RadM1-M3C04_G16_s20182811250279_e20182

...

writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/23/36/RadM2/OR_ABI-L1b-RadM2-M3C04_G16_s20182812336580_e20182812337038_c20182812337068.nc
writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/23/37/RadM2/OR_ABI-L1b-RadM2-M3C04_G16_s20182812337580_e20182812338038_c20182812338070.nc
writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/23/38/RadM2/OR_ABI-L1b-RadM2-M3C04_G16_s20182812338580_e20182812339038_c20182812339070.nc
writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/23/39/RadM2/OR_ABI-L1b-RadM2-M3C04_G16_s20182812339580_e20182812340038_c20182812340069.nc
writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/23/40/RadM2/OR_ABI-L1b-RadM2-M3C04_G16_s20182812340580_e20182812341038_c20182812341068.nc
writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/23/41/RadM2/OR_ABI-L1b-RadM2-M3C04_G16_s20182812341580_e20182812342038_c20182812342071.nc
writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/23/42/RadM2/OR_ABI-L1b-RadM2-M3C04_G16_s20182812342580_e20182

writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/23/32/RadM2/OR_ABI-L1b-RadM2-M3C05_G16_s20182812332580_e20182812333038_c20182812333079.nc
writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/23/33/RadM2/OR_ABI-L1b-RadM2-M3C05_G16_s20182812333580_e20182812334038_c20182812334084.nc
writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/23/34/RadM2/OR_ABI-L1b-RadM2-M3C05_G16_s20182812334580_e20182812335038_c20182812335083.nc
writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/23/35/RadM2/OR_ABI-L1b-RadM2-M3C05_G16_s20182812335580_e20182812336038_c20182812336080.nc
writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/23/36/RadM2/OR_ABI-L1b-RadM2-M3C05_G16_s20182812336580_e20182812337038_c20182812337081.nc
writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/23/37/RadM2/OR_ABI-L1b-RadM2-M3C05_G16_s20182812337580_e20182812338038_c20182812338083.nc
writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/23/38/RadM2/OR_ABI-L1b-RadM2-M3C05_G16_s20182812338580_e20182

writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/23/28/RadM2/OR_ABI-L1b-RadM2-M3C06_G16_s20182812328580_e20182812329043_c20182812329078.nc
writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/23/29/RadM2/OR_ABI-L1b-RadM2-M3C06_G16_s20182812329580_e20182812330045_c20182812330081.nc
writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/23/30/RadM2/OR_ABI-L1b-RadM2-M3C06_G16_s20182812330580_e20182812331043_c20182812331076.nc
writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/23/31/RadM2/OR_ABI-L1b-RadM2-M3C06_G16_s20182812331580_e20182812332043_c20182812332077.nc
writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/23/32/RadM2/OR_ABI-L1b-RadM2-M3C06_G16_s20182812332580_e20182812333043_c20182812333079.nc
writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/23/33/RadM2/OR_ABI-L1b-RadM2-M3C06_G16_s20182812333580_e20182812334043_c20182812334078.nc
writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/23/34/RadM2/OR_ABI-L1b-RadM2-M3C06_G16_s20182812334580_e20182

writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/23/24/RadM2/OR_ABI-L1b-RadM2-M3C07_G16_s20182812324580_e20182812325049_c20182812325080.nc
writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/23/25/RadM2/OR_ABI-L1b-RadM2-M3C07_G16_s20182812325580_e20182812326049_c20182812326082.nc
writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/23/26/RadM2/OR_ABI-L1b-RadM2-M3C07_G16_s20182812326580_e20182812327049_c20182812327082.nc
writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/23/27/RadM2/OR_ABI-L1b-RadM2-M3C07_G16_s20182812327580_e20182812328049_c20182812328083.nc
writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/23/28/RadM2/OR_ABI-L1b-RadM2-M3C07_G16_s20182812328580_e20182812329049_c20182812329081.nc
writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/23/29/RadM2/OR_ABI-L1b-RadM2-M3C07_G16_s20182812329580_e20182812330051_c20182812330083.nc
writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/23/30/RadM2/OR_ABI-L1b-RadM2-M3C07_G16_s20182812330580_e20182

writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/23/20/RadM2/OR_ABI-L1b-RadM2-M3C08_G16_s20182812320580_e20182812321038_c20182812321081.nc
writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/23/21/RadM2/OR_ABI-L1b-RadM2-M3C08_G16_s20182812321580_e20182812322038_c20182812322082.nc
writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/23/22/RadM2/OR_ABI-L1b-RadM2-M3C08_G16_s20182812322580_e20182812323038_c20182812323081.nc
writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/23/23/RadM2/OR_ABI-L1b-RadM2-M3C08_G16_s20182812323580_e20182812324038_c20182812324083.nc
writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/23/24/RadM2/OR_ABI-L1b-RadM2-M3C08_G16_s20182812324580_e20182812325038_c20182812325082.nc
writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/23/25/RadM2/OR_ABI-L1b-RadM2-M3C08_G16_s20182812325580_e20182812326038_c20182812326082.nc
writing file to /raid/tj/GOES/TEST/ABI-L1b-RadM/2018/281/23/26/RadM2/OR_ABI-L1b-RadM2-M3C08_G16_s20182812326580_e20182

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:

channels = [3,5,8]

print(days)

day = days[0]
n_channels = channels[0]
multivariate = False

if n_channels == 1:
    multivariate = False
    
model_path = '../saved-models/5Min-%iChannels/' % n_channels

flownet, interpnet, warper = inference_tools.load_models(n_channels,model_path, multivariate)

In [ ]:
def inference(block, flownet, interpnet, warper, multivariate):
    block_vals = torch.from_numpy(block.values)
    N = block_vals.shape[0]
    idxs = np.arange(0, N+1, 5)
    preds = []
    for idx0, idx1 in zip(idxs[:-1], idxs[1:]):
        idx1 = min(N-1, idx1)
        I0 = torch.unsqueeze(block_vals[idx0], 0).to(device)
        I1 = torch.unsqueeze(block_vals[idx1], 0).to(device)

        f = flownet(I0, I1)
        n_channels = I0.shape[1]
        
        if multivariate:
            f_01 = f[:,:2*n_channels]
            f_10 = f[:,2*n_channels:]
        else:
            f_01 = f[:,:2]
            f_10 = f[:,2:]

        T = idx1 - idx0 - 1
        predicted_frames = []
        for j in range(1,T+1):
            t = 1. * j / (T+1)
            I_t, g0, g1, V_t0, V_t1, delta_f_t0, delta_f_t1 = interpnet(I0, I1, f_01, f_10, t)
            predicted_frames.append(I_t.cpu().detach().numpy())
        
        preds += [I0.cpu().numpy()] + predicted_frames
            
    return block_predictions_to_dataarray(preds, block)


def testset_inference(n_channels, year, day, multivariate=False):
    model_path = '../saved-models/5Min-%iChannels/' % n_channels
    flownet, interpnet, warper = inference_tools.load_models(n_channels, model_path, False)
    if n_channels > 1:
        flownetmv, interpnetmv, warpermv = inference_tools.load_models(n_channels, model_path, True)
        
        
    dataset = goes16s3.NOAAGOESS3(product='ABI-L1b-RadM', 
                                  channels=range(1,n_channels+1))

    saved_data_files = []
    for houri, hour_das in enumerate(dataset.read_day(year, day)): # N,12or13,512,512,3
        blocked_data = utils.blocks(hour_das, width=352)
        sv_interpolated_hour, mv_interpolated_hour = [], []
        for block_num, block in enumerate(blocked_data):            
            sv_da = inference(block, flownet, interpnet, warper, False)
            sv_interpolated_hour.append(sv_da)
            if n_channels > 1:
                mv_da = inference(block, flownetmv, interpnetmv, warpermv, True)
                mv_interpolated_hour.append(mv_da)

        sv_prediction_da = merge_and_average_dataarrays(sv_interpolated_hour)
        ds = xr.Dataset({'observed': hour_das, 'sv_predicted': sv_prediction_da})
        if n_channels > 1:
            ds['mv_predicted'] = merge_and_average_dataarrays(mv_interpolated_hour)
        ncpath = os.path.join(model_path, '%04i_%03i' % (year, day))
        if not os.path.exists(ncpath):
            os.mkdir(ncpath)
        ncfile = os.path.join(ncpath,  '%04i_%03i_%02i.nc' % (year, day, houri))

        print("Saved to file: {}".format(ncfile))
        ds.to_netcdf(ncfile)
        saved_data_files.append(ncfile)
        
#for c in channels:
for c in [3,5,8]:
    for day in days:
        print("Test inference for channel {}, year 2018, and day {}".format(c, day))
        testset_inference(c, 2018, day)

In [ ]:
def psnr(img1, img2, axis=None):
    img1[img1 == np.inf] = np.nan
    img2[img2 == np.inf] = np.nan
    
    mse = np.nanmean((img1 - img2) ** 2, axis=axis)
    
    #print('obs', np.histogram(img2.flatten(),
    #                range=[np.nanmin(img2), np.nanmax(img2)]))
    #if isinstance(mse, float) and mse == 0:
    #    return 100
    if np.any(mse[mse == np.inf]):
        return np.zeros_like(mse)
    
    #if isinstance(mse, float) and mse == np.inf:
    #    return None

    PIXEL_MAX = 1.0
    return 20 * np.log10(PIXEL_MAX / np.sqrt(mse))


def ssim(img1, img2):
    img1[img1 == np.inf] = np.nan
    img2[img2 == np.inf] = np.nan
    
    img1[np.isnan(img1)] = 0.
    img2[np.isnan(img2)] = 0.
    r = []
    for b in range(img1.shape[1]):
        sms = [compare_ssim(img1[i,b], img2[i,b]) for i in range(img1.shape[0])]
        r.append(np.nanmean(sms))
    return np.array(r)

#def bandwise_psnr(img1, img2): # assuming (t,c,h,w)
    

In [ ]:
results = []
for c in [1,3,5,8]:
    model_path = './saved-models/5Min-%iChannels/2018_199/' % c
    saved_data_files = sorted([os.path.join(model_path, f) 
                        for f in os.listdir(model_path) if f[-3:] == '.nc'])
    for sf in saved_data_files:
        print(sf)
        ds = xr.open_dataset(sf)        
        svpsnr = psnr(ds.sv_predicted.values, ds.observed.values, (3,2,0))
        svssim = ssim(ds.sv_predicted.values, ds.observed.values)
        if c > 1:
            mvpsnr = psnr(ds.mv_predicted.values, ds.observed.values, (3,2,0))
            mvssim = ssim(ds.mv_predicted.values, ds.observed.values)
        else:
            mvpsnr = [None]*len(svpsnr)
            mvssim = [None]*len(svpsnr)

        for b in range(len(svpsnr)):
            r = {'Channels': c, 'Band': b+1, 'Model': 'A-Single', 'PSNR': svpsnr[b], 'SSIM': svssim[b]}
            results.append(r)
            r = {'Channels': c, 'Band': b+1, 'Model': 'B-Multi', 'PSNR': mvpsnr[b], 'SSIM': mvssim[b]}
            results.append(r)

In [ ]:
results_df = pd.DataFrame(results)
results_df.set_index(["Channels", "Band", "Model"])
psnr_per_band_table = pd.pivot_table(results_df, values='PSNR', index=['Band'], 
                       columns=['Channels', 'Model'])

print(psnr_per_band_table)
reslatex = psnr_per_band_table.to_latex(float_format='{:,.2f}'.format)
print(reslatex)


results_df = pd.DataFrame(results)
results_df.set_index(["Channels", "Band", "Model"])
psnr_per_band_table = pd.pivot_table(results_df, values='PSNR', index=['Channels'], 
                       columns=['Model'])

print(psnr_per_band_table)
reslatex = psnr_per_band_table.to_latex(float_format='{:,.2f}'.format)
print(reslatex)


In [ ]:
ssim_per_band_table = pd.pivot_table(results_df, values='SSIM', index=['Band'], 
                       columns=['Channels', 'Model'])

print(ssim_per_band_table)
reslatex = ssim_per_band_table.to_latex(float_format='{:,.3f}'.format)
print(reslatex)

#### Figure 1: Dataset Examples
###  1 Day - Full Disk Coverage, CONUS, and MESOSCALE


In [ ]:

month = 3
day = 3
year = 2018
hour = 20
channels = [1,2,3]
#channels = [10]

day_of_year = datetime(year, month, day).timetuple().tm_yday

products = ['ABI-L1b-RadF', 'ABI-L1b-RadC', 'ABI-L1b-RadM']

arrs = []
for p in products:
    prod = goes16s3.NOAAGOESS3(product=p, 
                               channels=channels)
    prod_day_keys = prod.day_keys(year, day_of_year, hours=[hour])
    prod_key = prod_day_keys.keyname[0]
    minute = prod_day_keys.minute.min()
    prod_minute_keys = prod_day_keys[prod_day_keys.minute == minute]

    das = []
    for c in channels:
        k = prod_minute_keys[prod_minute_keys.channel == c].keyname.values[0]
        ds, _ = prod.read_nc_from_s3(k)
        da = ds.Rad
        if c == 2:
            da = utils.interp_da2d(da, 1./4, fillna=False)
        elif c in [1,3,5]:
            da = utils.interp_da2d(da, 1./2, fillna=False)
        das.append(da)

    arr = np.concatenate([d.values[np.newaxis] for d in das], axis=0)
    arrs.append(arr)

In [ ]:
filenames = ['fulldisk.png', 'conus.png', 'mesoscale.png']
for i, arr in enumerate(arrs):
    plt.imsave('figures/' + filenames[i],
               arr.transpose(1,2,0)[:,:,[1,2,0]],
               dpi=50)
    

## Figure 2: Interpolation Network

In [ ]:
prod = goes16s3.NOAAGOESS3(product='ABI-L1b-RadM', channels=channels)

ds = prod.read_day(year, day, hours=[hour]).next()

In [ ]:
blocked_data = utils.blocks(hour_das, width=352)
B = blocked_data[0]

I0_np = B.isel(t=0).values
I0 = torch.from_numpy(I0_np[np.newaxis]).to(device)
I1_np = B.isel(t=15).values
I1 = torch.from_numpy(I1_np[np.newaxis]).to(device)


print(I0.shape, I1.shape)

f = flow_net(I0, I1)

# x, y optical flows
f_10 = f[:,:2]
f_01 = f[:,2:]

T = 4
i = 2
t = 1. * i / (T+1)
# Input Channels: predicted image and warped without derivatives
I_t, g0, g1 = interp_net(I0, I1, f_10, f_01, t)

In [ ]:
obs = B.isel(t=i+1).values
pred = I_t.cpu().detach().numpy()

# Images to save
# I0
# I1 

def make_img(I, f, vmin=None, vmax=None):
    plt.imshow(I, vmin=vmin, vmax=vmax)
    plt.axis('off')
    plt.show()
    scipy.misc.imsave(f, I)
    
make_img(I0_np[[1,2,0]].transpose(1,2,0), 'figures/I0.png')
make_img(I1_np[[1,2,0]].transpose(1,2,0), 'figures/I1.png')
make_img((I1_np-I0_np)[[1,2,0]].transpose(1,2,0), 'figures/I1-minus-I0.png')

make_img(obs[[1,2,0]].transpose(1,2,0), 'figures/IT.png')


f_10_np = f_10.cpu().detach().numpy()
f_01_np = f_01.cpu().detach().numpy()

#mn = np.percentile(f_10_np[0,1], 10.)
#mx = np.percentile(f_10_np[0,1], 90.)

f_10_np = f_10_np.mean(axis=(0,1))
make_img(f_10_np[20:330,20:330], 'figures/f_10.png')#, vmin=mn, vmax=mx)

f_01_np = f_01_np.mean(axis=(0,1))
make_img(f_01_np[20:330,20:330], 'figures/f_01.png')#, vmin=mn, vmax=mx)

print(g0.shape)

## Flow Figures

In [ ]:
dataset = goes16s3.NOAAGOESS3(product='ABI-L1b-RadM', 
                          channels=range(1,4))

print(datetime(2018,3,3))
hour_das = dataset.read_day(year, datetime(2018, 3, 3).timetuple().tm_yday, 
                            hours=[20]).next() # N,12or13,512,512,3
blocked_data = utils.blocks(hour_das, width=352)

In [ ]:
def opticalflow(flow):
    hsv = np.ones((flow.shape[0], flow.shape[1], 3))*255.
    
    # Use Hue, Saturation, Value colour model 
    mag, ang = cv2.cartToPolar(flow[:,:,0], flow[:,:,1])
    
    hsv[:,:, 0] = ang * 180 / np.pi / 2
    hsv[:,:, 2] = cv2.normalize(mag, None, 0, 255., cv2.NORM_MINMAX)
    bgr = cv2.cvtColor(np.uint8(hsv), cv2.COLOR_HSV2BGR)
    return bgr

def flowfigures(block):
    n_channels = 3
    model_path = './saved-models/5Min-3Channels/'
    flownet, interpnet, warper = load_models(n_channels, model_path, False)
        
    block_vals = torch.from_numpy(block.values)
    N = block_vals.shape[0]
    idxs = np.arange(0, N+1, 5)
    preds = []
    
    idx0 = 0
    idx1 = 5

    j = 2
    t = 1. * j / 5
    
    I0 = torch.unsqueeze(block_vals[idx0], 0).to(device)
    I1 = torch.unsqueeze(block_vals[idx1], 0).to(device)
    IT_img = block_vals[idx0 + j].detach().cpu().numpy().transpose(1,2,0)
    
    f = flownet(I0, I1)

    f_01 = f[:,:2]
    f_10 = f[:,2:]

    I_t, g0, g1, V_t0, V_t1, delta_f_t0, delta_f_t1 = interpnet(I0, I1, f_01, f_10, t)
    F_t0_hat = -(1-t) * t * f_01 + t**2 * f_10
    F_t1_hat =  (1-t) ** 2 * f_01 - t * (1 - t) * f_10
    F_t0 = F_t0_hat + delta_f_t0
    F_t1 = F_t1_hat + delta_f_t1
    
    
    detach = lambda x: x.detach().cpu().numpy()[0].transpose(1,2,0)
    
    I0_img = detach(I0)
    I1_img = detach(I1)
    plt.imsave('figures/flowfigs/I0.png', I0_img)
    plt.imsave('figures/flowfigs/I1.png', I1_img)

    
    V_t0_img = detach(V_t0)
    plt.imsave('figures/flowfigs/V_t0.png', V_t0_img[:,:,0], cmap='gray')
    V_t1_img = detach(V_t1)
    plt.imsave('figures/flowfigs/V_t1.png', V_t1_img[:,:,0], cmap='gray')
    plt.imsave('figures/flowfigs/V_tdiff.png', V_t1_img[:,:,0] - V_t0_img[:,:,0], cmap='gray')

    f_01_img = opticalflow(detach(F_t0)[20:332,30:332])
    plt.imsave('figures/flowfigs/F_t0.png', f_01_img)
    f_10_img = opticalflow(detach(F_t1)[20:332,30:332])
    plt.imsave('figures/flowfigs/F_t1.png', f_10_img)
    
    I_t_img = detach(I_t)
    plt.imsave('figures/flowfigs/I_t.png', I_t_img)
    diff = IT_img - I_t_img
    plt.imsave('figures/flowfigs/residual.png', diff*5)
    
    
flowfigures(blocked_data[0])

In [ ]:
## Average PSNR 